# Inbalanced Classes
## In this lab, we are going to explore a case of imbalanced classes. 


Like we disussed in class, when we have noisy data, if we are not careful, we can end up fitting our model to the noise in the data and not the 'signal'-- the factors that actually determine the outcome. This is called overfitting, and results in good results in training, and in bad results when the model is applied to real data. Similarly, we could have a model that is too simplistic to accurately model the signal. This produces a model that doesnt work well (ever). 


### Note: before doing the first commit, make sure you don't include the large csv file, either by adding it to .gitignore, or by deleting it.

### First, download the data from: https://www.kaggle.com/ntnu-testimon/paysim1. Import the dataset and provide some discriptive statistics and plots. What do you think will be the important features in determining the outcome?
### Note: don't use the entire dataset, use a sample instead, with n=100000 elements, so your computer doesn't freeze.

In [1]:
# Your code here
import pandas as pd
import zipfile

zf = zipfile.ZipFile('archive.zip') 

fraud_detection = pd.read_csv(zf.open('PS_20174392719_1491204439457_log.csv'), header=0, encoding='ISO-8859-1')
fraud_detection.head()


In [ ]:
sample_df = fraud_detection.sample(100000)
sample_df.describe()

In [ ]:
sample_df.info()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(16, 6))
mask = np.triu(np.ones_like(sample_df.corr(), dtype=np.bool))
heatmap = sns.heatmap(sample_df.corr(), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Triangle Correlation Heatmap', fontdict={'fontsize':18}, pad=16)

### What is the distribution of the outcome? 

In [ ]:
# Your response here
import numpy as np
import matplotlib.pyplot as plt

pd.value_counts(sample_df['isFraud']).plot(kind = 'bar')

### Clean the dataset. How are you going to integrate the time variable? Do you think the step (integer) coding in which it is given is appropriate?

In [1]:
# Your code here
pd.value_counts(sample_df['type']).plot(kind = 'bar')

In [ ]:
sample_df = pd.get_dummies(sample_df, columns=['type'])
sample_df.drop(['nameOrig','nameDest'], axis = 1, inplace=True)
sample_df.head()

### Run a logisitc regression classifier and evaluate its accuracy.

In [ ]:
# Your code here
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

data = sample_df.drop(labels='isFraud', axis=1)
target = sample_df['isFraud']

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size= 0.2)

LR = LogisticRegression()
LR.fit(X_train,y_train)
LR.score(X_test,y_test)
pred = LR.predict(X_test)

LR.score(X_train,y_train)

In [ ]:
pred = LR.predict(X_test)
confusion_matrix(y_test, pred)

In [ ]:

display(accuracy_score(y_train, LR.predict(X_train)))
display(accuracy_score(y_test, LR.predict(X_test)))

### Now pick a model of your choice and evaluate its accuracy.

In [ ]:
# Your code here
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

clf = GradientBoostingClassifier(n_estimators = 1000
                                ,learning_rate = 1
                                ,max_depth = 1
                                ,random_state = 0).fit(X_train, y_train)

display(clf.score(X_train,y_train))

display(accuracy_score(y_train, clf.predict(X_train)))
display(accuracy_score(y_test, clf.predict(X_test)))

display(confusion_matrix(y_test, clf.predict(X_test)))

### Which model worked better and how do you know?

In [2]:
# Your response here

### Note: before doing the first commit, make sure you don't include the large csv file, either by adding it to .gitignore, or by deleting it.